In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
daycount(30, "offshore", "US", 0.0487)

0.004058333333333334

## Money Markets

Daycount Conventions

In [35]:
def daycount(days, market, country, r):
    if market == "offshore" and country in ["US", "JPY", "EU", "CHF"]:
        res = r * days/360
        return res
    elif market == "offshore" and country == "GBP": 
        res = r*days/365
        return res 
    elif market == "domestic" and country == "JPY":
        res = r * days/365
        return res
    else:
        res = (r * days)/360
        return res

T-Bill Pricer

In [42]:
def TBillPricer(faceval, yld, days, market, country):
    
    price = faceval * (1 - daycount(days, market, country, yld))
    
    return price

In [39]:
TBillPricer(faceval = 20000000, 
         yld = 0.0487, 
         days = 30,
         market = "domestic", 
         country = "US")

19918833.333333332

Certificate of Deposit Pricer

In [40]:
def CDPricer(facval, rate,days, market, country):
    price = facval * (1 + daycount(days, market, country, rate))
    return price 

In [41]:
CDPricer(facval = 2000000,
         coupon =  0.0575,
         days = 90, 
         market = "offshore", 
         country = "US")

2040000.0

Pricer for a CD that has already been initialized. 

In [49]:
def CertDepositPricer(initial_r, facval, current_r ,int_day,buy_day, market, country):
    
    up = (facval * (1 + daycount(int_day, market, country, initial_r)))
    
    down = (1 + daycount(buy_day, market, country, current_r))
    
    price = up/down
    
    diff = up - price
    
    return price, diff

In [50]:
CertDepositPricer(initial_r = 0.0575,
                  facval = 2000000,
                  current_r =  0.0525,
                  int_day = 90, 
                  buy_day = 30, 
                  market = "offshore", 
                  country = "US")

(2019912.8811449907, 8837.118855009321)

#### Discounted Securities

European Commerical Paper

In [56]:
def ECPricer(faceval, rate, days, market, country):
    price = faceval/ (1 + daycount(days, market, country, rate))
    return price

In [58]:
ECPricer(faceval = 20000000, 
         rate = 0.0526, 
         days = 30,
         market = "offshore", 
         country = "US")

19912715.928513348

US Commercial Paper and T-Bills

In [9]:
def USCP_TBillsPricer(faceval, rate, days, market, country):
    price = faceval * (1 - daycount(days, market, country, rate))
    return price

In [10]:
USCP_TBillsPricer(faceval = 1000000, 
         rate = 0.10, 
         days = 90,
         market = "offshore", 
         country = "US")

970000.0

Money-Market Return

In [12]:
def MMReturn(FV, price, days, market, country):
    side_one = (FV - price) / price 
    side_two = daycount(days, market, country, 0)
    res = side_one * side_two
    return res

In [13]:
MMReturn(FV = 1000000,
         price = FV,
         days = 90,
         market = "domestic",
         country = "US") 

0.00510204081632653

### Exercises

In [61]:
t_bill = TBillPricer(faceval = 20000000, 
         yld = 0.0487, 
         days = 30,
         market = "domestic", 
         country = "US")
20000000 + ((t_bill - 20000000) * -1)

20081166.666666668

## Debt Markets and DCM

In [66]:
def accrued_intrest(convention, notional, coupon, days):
    if convention == "Act/Act":
        acc_int = notional * coupon * days/365
        return acc_int
    else:
        acc_int = notional * coupon * days/360

In [69]:
accrued_intrest(
    convention = "Act/Act", 
    notional = 100000000000, 
    coupon = 0.01125, 
    days = 109)

335958904.10958904

In [75]:
def dirty_price(convention, notional, coupon, days):
    acc_int = notional * coupon * days/365
    dirty_price = notional + acc_int 
    dirty_price = (dirty_price /notional ) *100
    return dirty_price

In [76]:
dirty_price(
    convention = "Act/Act", 
    notional = 100000000000, 
    coupon = 0.01125, 
    days = 109)

100.33595890410957

In [77]:
import math
 
def total_present_value(face_value, coupon, periods, rate):
    total_pv = 0
    for n in range(1, periods+1):
        total_pv += coupon / math.pow((1 + rate), n)
 
    total_pv += face_value / math.pow((1 + rate), periods)
 
    return total_pv

In [88]:
def YTM(price, coupon, annuality, remaining_years,):  
   
    ytm = coupon
    condition = True
    
    while condition:
        if (price < 100):
            ytm += 0.0000001
        else:
            ytm -= 0.0000001
 
        total_pv = total_present_value(100, coupon/annuality, remaining_years *annuality, ytm/annuality)
 
        if (price < 100):
            condition = total_pv > price
        else:
            condition = total_pv < price
 
    print("Yield to Maturity:  %.2f%%" % (ytm*100.0))

In [89]:
YTM(
    price = 98.63, 
    coupon = 0.04,
    annuality = 1,
    remaining_years = 3, 
   )

Yield to Maturity:  4.00%
